In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts


from mne_connectivity import spectral_connectivity_epochs
from mne.datasets import sample
import seaborn as sns

from scipy.signal import welch
import yasa
import constants
import numpy as np

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [65]:
data_type = 'REM'
data_type = 'N3'

#['REM', 'N1', 'N2', 'N3'. 'Wake']
power_bands = {'delta' : (0.5,4) , 'theta' : (4,8) , 'alpha' : (8,12) , 'sigma' : (12,16) , 'beta' : (16,30) , 'gamma' : (30,40) }
power_band_names = [val for val in power_bands.keys()]
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/Connectivity/'
X = pd.read_hdf(folder + data_type+ '_coherence_df.h5')
X = X.reset_index(drop = True)
y = pd.read_hdf('/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/C_22_data/Full_dfs/' + data_type + '_y.h5')
groups = pd.read_hdf('/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/C_22_data/Full_dfs/' + data_type + '_groups.h5')

## Adjust code to run expts properly

In [71]:
subsets_dict = {}
for p_band in power_band_names:
    subsets_dict[p_band] = [col for col in X.columns if p_band in col]
subsets_dict['all_features'] = [col for col in X.columns]

In [86]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

# Expt 1

In [88]:
t1 = time.time()

clfs_dict =  {'RF' : RandomForestClassifier() , 'Ada_B' : AdaBoostClassifier() , 'SVM' : SVC()} 

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1)
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, clfs_dict , return_df = True , subset_names_and_cols = subsets_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)   

t2 = time.time()

t2-t1

print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0) )

res_df

delta_RF_acc              0.606944
delta_Ada_B_acc           0.608333
delta_SVM_acc             0.629167
theta_RF_acc              0.618056
theta_Ada_B_acc           0.631944
theta_SVM_acc             0.629167
alpha_RF_acc              0.555556
alpha_Ada_B_acc           0.527778
alpha_SVM_acc             0.629167
sigma_RF_acc              0.663889
sigma_Ada_B_acc           0.522222
sigma_SVM_acc             0.629167
beta_RF_acc               0.556944
beta_Ada_B_acc            0.612500
beta_SVM_acc              0.629167
gamma_RF_acc              0.525000
gamma_Ada_B_acc           0.536111
gamma_SVM_acc             0.629167
all_features_RF_acc       0.570833
all_features_Ada_B_acc    0.479167
all_features_SVM_acc      0.629167
dtype: float64


,random_state,delta_RF_acc,delta_RF_std,delta_Ada_B_acc,delta_Ada_B_std,delta_SVM_acc,delta_SVM_std,theta_RF_acc,theta_RF_std,theta_Ada_B_acc,...,gamma_Ada_B_acc,gamma_Ada_B_std,gamma_SVM_acc,gamma_SVM_std,all_features_RF_acc,all_features_RF_std,all_features_Ada_B_acc,all_features_Ada_B_std,all_features_SVM_acc,all_features_SVM_std
0,1,0.561111,0.167728,0.608333,0.056519,0.627778,0.240145,0.536111,0.145562,0.652778,...,0.563889,0.175814,0.627778,0.240145,0.536111,0.165645,0.561111,0.125031,0.627778,0.240145
1,2,0.652778,0.169649,0.608333,0.097183,0.630556,0.142346,0.700000,0.203101,0.611111,...,0.508333,0.214735,0.630556,0.142346,0.605556,0.151790,0.397222,0.245578,0.630556,0.142346


# Expt 2

In [92]:
t1 = time.time()

clfs_dict =  {'RF' : RandomForestClassifier() , 'Ada_B' : AdaBoostClassifier() , 'SVM' : SVC()} 


X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2)
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, clfs_dict , return_df = True , subset_names_and_cols = subsets_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)   

t2 = time.time()

t2-t1

print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0) )

res_df


delta_RF_acc              0.532222
delta_Ada_B_acc           0.596667
delta_SVM_acc             0.450000
theta_RF_acc              0.418889
theta_Ada_B_acc           0.457778
theta_SVM_acc             0.388889
alpha_RF_acc              0.460000
alpha_Ada_B_acc           0.416667
alpha_SVM_acc             0.378889
sigma_RF_acc              0.450000
sigma_Ada_B_acc           0.494444
sigma_SVM_acc             0.436667
beta_RF_acc               0.440000
beta_Ada_B_acc            0.552222
beta_SVM_acc              0.421111
gamma_RF_acc              0.450000
gamma_Ada_B_acc           0.501111
gamma_SVM_acc             0.387778
all_features_RF_acc       0.463333
all_features_Ada_B_acc    0.502222
all_features_SVM_acc      0.388889
dtype: float64


,random_state,delta_RF_acc,delta_RF_std,delta_Ada_B_acc,delta_Ada_B_std,delta_SVM_acc,delta_SVM_std,theta_RF_acc,theta_RF_std,theta_Ada_B_acc,...,gamma_Ada_B_acc,gamma_Ada_B_std,gamma_SVM_acc,gamma_SVM_std,all_features_RF_acc,all_features_RF_std,all_features_Ada_B_acc,all_features_Ada_B_std,all_features_SVM_acc,all_features_SVM_std
0,1,0.564444,0.172905,0.566667,0.153639,0.462222,0.099032,0.462222,0.171932,0.477778,...,0.524444,0.159536,0.400000,0.131280,0.464444,0.144598,0.544444,0.214073,0.400000,0.131280
1,2,0.500000,0.035136,0.626667,0.032660,0.437778,0.119588,0.375556,0.051831,0.437778,...,0.477778,0.237684,0.375556,0.103375,0.462222,0.133444,0.460000,0.132330,0.377778,0.096097


# Expt 3

In [93]:
t1 = time.time()

clfs_dict =  {'RF' : RandomForestClassifier() , 'Ada_B' : AdaBoostClassifier() , 'SVM' : SVC()} 


X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 3)
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, clfs_dict , return_df = True , subset_names_and_cols = subsets_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)   

t2 = time.time()

t2-t1

print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0) )

res_df

delta_RF_acc              0.401905
delta_Ada_B_acc           0.297619
delta_SVM_acc             0.284286
theta_RF_acc              0.347143
theta_Ada_B_acc           0.384762
theta_SVM_acc             0.242381
alpha_RF_acc              0.303333
alpha_Ada_B_acc           0.250000
alpha_SVM_acc             0.257619
sigma_RF_acc              0.341429
sigma_Ada_B_acc           0.369524
sigma_SVM_acc             0.340000
beta_RF_acc               0.330952
beta_Ada_B_acc            0.430000
beta_SVM_acc              0.257143
gamma_RF_acc              0.299048
gamma_Ada_B_acc           0.353810
gamma_SVM_acc             0.250952
all_features_RF_acc       0.382857
all_features_Ada_B_acc    0.339524
all_features_SVM_acc      0.271905
dtype: float64


,random_state,delta_RF_acc,delta_RF_std,delta_Ada_B_acc,delta_Ada_B_std,delta_SVM_acc,delta_SVM_std,theta_RF_acc,theta_RF_std,theta_Ada_B_acc,...,gamma_Ada_B_acc,gamma_Ada_B_std,gamma_SVM_acc,gamma_SVM_std,all_features_RF_acc,all_features_RF_std,all_features_Ada_B_acc,all_features_Ada_B_std,all_features_SVM_acc,all_features_SVM_std
0,1,0.413333,0.135807,0.231429,0.153968,0.263810,0.080902,0.302857,0.113273,0.448571,...,0.304762,0.091894,0.251429,0.061677,0.320952,0.076047,0.334286,0.124555,0.292381,0.083549
1,2,0.390476,0.062886,0.363810,0.119128,0.304762,0.066052,0.391429,0.169176,0.320952,...,0.402857,0.052216,0.250476,0.069830,0.444762,0.130431,0.344762,0.141280,0.251429,0.041991


# Expt 4

In [94]:
t1 = time.time()

clfs_dict =  {'RF' : RandomForestClassifier() , 'Ada_B' : AdaBoostClassifier() , 'SVM' : SVC()} 

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 4)
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, clfs_dict , return_df = True , subset_names_and_cols = subsets_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)   

t2 = time.time()

t2-t1

print(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0) )

res_df

delta_RF_acc           0.367143
theta_RF_acc           0.276190
alpha_RF_acc           0.317143
sigma_RF_acc           0.334286
beta_RF_acc            0.257143
gamma_RF_acc           0.277619
all_features_RF_acc    0.326667
dtype: float64


,random_state,delta_RF_acc,delta_RF_std,delta_Ada_B_acc,delta_Ada_B_std,delta_SVM_acc,delta_SVM_std,theta_RF_acc,theta_RF_std,theta_Ada_B_acc,...,gamma_Ada_B_acc,gamma_Ada_B_std,gamma_SVM_acc,gamma_SVM_std,all_features_RF_acc,all_features_RF_std,all_features_Ada_B_acc,all_features_Ada_B_std,all_features_SVM_acc,all_features_SVM_std
0,1,0.386667,0.070334,0.329524,0.130528,0.263810,0.080902,0.274286,0.122617,0.279048,...,0.309524,0.162631,0.251429,0.099632,0.347619,0.120091,0.251429,0.099632,0.278095,0.100140
1,2,0.347619,0.062886,0.429524,0.098809,0.317143,0.128257,0.278095,0.009331,0.336190,...,0.322857,0.128850,0.262857,0.045216,0.305714,0.093721,0.262857,0.045216,0.304762,0.087495


In [95]:
print(res_df[[col for col in res_df.columns if 'acc' in col]].mean(axis = 0) )

delta_RF_acc              0.367143
delta_Ada_B_acc           0.379524
delta_SVM_acc             0.290476
theta_RF_acc              0.276190
theta_Ada_B_acc           0.307619
theta_SVM_acc             0.249048
alpha_RF_acc              0.317143
alpha_Ada_B_acc           0.235238
alpha_SVM_acc             0.284286
sigma_RF_acc              0.334286
sigma_Ada_B_acc           0.293810
sigma_SVM_acc             0.319524
beta_RF_acc               0.257143
beta_Ada_B_acc            0.340476
beta_SVM_acc              0.298571
gamma_RF_acc              0.277619
gamma_Ada_B_acc           0.316190
gamma_SVM_acc             0.257143
all_features_RF_acc       0.326667
all_features_Ada_B_acc    0.257143
all_features_SVM_acc      0.291429
dtype: float64


## RBD vs Others experiments

In [19]:
y_expt.value_counts()

0    26
3    22
dtype: int64

In [96]:
# class_label_dict = {'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3} Dictionary used to label the classes for reference

#First lets try RBD vs HC 
y_expt = y.loc[ (y==0) | (y ==3)]
X_expt = X.loc[y_expt.index]
groups_expt = groups.loc[y_expt.index]

#Run it 
t1 = time.time()

for power_band in list(power_bands.keys()) +['_']:
    print(power_band)
    acc_scores = []
    X_subset = X_expt[[col for col in X_expt.columns if power_band in col]]
    for i in range(10):
        clf = RandomForestClassifier()
        X_train, X_test, y_train, y_test = train_test_split(X_subset, y_expt, test_size=0.2)
        
        clf.fit(X_train, y_train) 
        y_pred = clf.predict(X_test)
        acc_scores.append(str(accuracy_score(y_test, y_pred ))[:4] )
    
    print(acc_scores)    
    print(np.array([float(acc) for acc in acc_scores]).mean())

t2 = time.time()

t2-t1

delta
['1.0', '0.7', '0.6', '0.6', '0.7', '0.3', '0.7', '0.5', '0.6', '0.9']
0.6599999999999999
theta
['0.9', '0.5', '0.5', '0.6', '0.6', '0.8', '0.7', '0.7', '0.5', '0.8']
0.6599999999999999
alpha
['0.8', '0.6', '0.7', '0.6', '0.3', '0.6', '0.5', '0.6', '0.8', '0.8']
0.6299999999999999
sigma
['0.4', '0.6', '0.6', '0.5', '0.8', '0.5', '0.7', '0.5', '0.8', '0.4']
0.58
beta
['0.6', '0.2', '0.6', '0.5', '0.3', '0.5', '0.6', '0.7', '0.3', '0.7']
0.5
gamma
['0.5', '0.7', '0.6', '0.6', '0.6', '0.7', '0.5', '0.6', '0.5', '0.6']
0.59
_
['0.9', '0.5', '0.8', '0.8', '0.8', '0.7', '0.6', '0.7', '0.7', '0.6']
0.71


10.311355352401733

In [64]:
y_temp.value_counts()

# class_label_dict = {'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3} Dictionary used to label the classes for reference

#First lets try RBD vs HC 

#Label PD & HC in the same category 
y_temp = y.copy()
# y_temp[y==1]= 0 
y_temp[y==2]= 3

y_expt = y_temp.loc[ (y_temp==0) | (y_temp==3)]
X_expt = X.loc[y_expt.index]
groups_expt = groups.loc[y_expt.index]

#Run it 
t1 = time.time()

for power_band in list(power_bands.keys()) +['_']:
    print(power_band)
    acc_scores = []
    X_subset = X_expt[[col for col in X_expt.columns if power_band in col]]
    for i in range(10):
        clf = RandomForestClassifier()
        X_train, X_test, y_train, y_test = train_test_split(X_subset, y_expt, test_size=0.2)
        
        clf.fit(X_train, y_train) 
        y_pred = clf.predict(X_test)
        acc_scores.append(str(accuracy_score(y_test, y_pred ))[:4] )
    
    print(acc_scores)    
    print(np.array([float(acc) for acc in acc_scores]).mean())

t2 = time.time()

t2-t1

delta
['0.66', '0.83', '0.75', '0.83', '0.58', '0.58', '0.75', '0.83', '0.66', '0.83']
0.7300000000000001
theta
['0.66', '0.66', '0.75', '0.83', '0.66', '0.58', '0.75', '0.66', '0.41', '0.66']
0.6620000000000001
alpha
['0.5', '0.5', '0.66', '0.5', '0.58', '0.66', '0.58', '0.58', '0.66', '0.5']
0.5720000000000001
sigma
['0.58', '0.66', '0.33', '0.75', '0.58', '0.16', '0.58', '0.66', '0.41', '0.75']
0.546
beta
['0.66', '0.58', '0.33', '0.58', '0.58', '0.75', '0.58', '0.41', '0.58', '0.66']
0.5710000000000001
gamma
['0.66', '0.41', '0.66', '0.5', '0.58', '0.58', '0.5', '0.75', '0.66', '0.33']
0.5630000000000001
_
['0.66', '0.75', '0.33', '0.75', '0.5', '0.5', '0.75', '0.83', '0.91', '0.75']
0.673


10.735900163650513